In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
import pandas as pd

In [29]:
df_clusters = pd.read_excel("/content/gdrive/My Drive/Colab Notebooks/Pipeline_Extended/topic_clusters_labeled.xlsx", sheet_name=0, na_values="NaN")
df_topics = pd.read_excel("/content/gdrive/My Drive/Colab Notebooks/Pipeline_Extended/output_bert_topics_df_withStopwords_v2.xlsx", sheet_name=0, na_values="NaN")
df_problem_solution = pd.read_excel("/content/gdrive/My Drive/Colab Notebooks/Pipeline_Extended/output_user_agent_no_names_plus_topics.xlsx", sheet_name=0, na_values="NaN")

In [1]:
#Merge Problem-Solution Excel with Topics excel
df = pd.merge(df_problem_solution,df_topics[['number','topic']],on='number', how='left')

NameError: name 'pd' is not defined

In [2]:
#Then merge with clusters
df = pd.merge(df,df_clusters[['topic','merged_clusters','Filter']],on='topic', how='left')

NameError: name 'pd' is not defined

In [3]:
new_column = []
for values in df['problem']:
    clean_value = re.sub(r"[^a-zA-Z0-9 ]","", values)
    clean_value = clean_value.replace('Additional Location', '') 
    clean_value = clean_value.replace('Additional comments Information', '')
    clean_value = clean_value.replace('Affected Device', '')
    clean_value = clean_value.replace('Issue Description', '')
    clean_value = clean_value.replace('Issue Description', '')
    clean_value = re.sub(r"name[0-9]+", "", values)
    clean_value_split = " ".join(clean_value.split())
    new_column.append(len(clean_value_split.split()))

df['problem_word_count'] = new_column

new_column = []
for values in df['solution']:
    clean_value = re.sub(r"[^a-zA-Z0-9 ]","", values)
    clean_value = clean_value.replace('Additional Location', '') 
    clean_value = clean_value.replace('Additional comments Information', '')
    clean_value = clean_value.replace('Affected Device', '')
    clean_value = clean_value.replace('Issue Description', '')
    clean_value = clean_value.replace('Issue Description', '')
    clean_value = re.sub(r"name[0-9]+", "", values)
    clean_value_split = " ".join(clean_value.split())
    new_column.append(len(clean_value_split.split()))
    
df['solution_word_count'] = new_column

NameError: name 'df' is not defined

In [49]:
df2 = df["problem_word_count"].mean()
print(df2)
df3 = df["solution_word_count"].mean()
print(df3)

22.48552202587426
72.45230828780277


In [50]:
df.head()
df.shape[0]

67171

In [22]:
#df.to_excel("/content/gdrive/My Drive/Colab Notebooks/Pipeline_Extended/output_user_agent_no_names_plus_topics_clusters.xlsx")

In [77]:
df_filtered = df[(df.problem_word_count >5) & (df.solution_word_count >35)]

In [60]:
df_filtered.to_excel("/content/gdrive/My Drive/Colab Notebooks/Pipeline_Extended/output_user_agent_no_names_plus_topics_clusters_filtered_on_word_count_v2.xlsx")

In [79]:
df_filtered.shape[0]

26865

In [80]:
#Filter on true
df_filtered = df_filtered.loc[df['Filter']==True]

In [81]:
print("Filtered on cluster quality:")
df_filtered.shape[0]

Filtered on cluster quality:


8187

In [82]:
df_filtered = df_filtered.groupby('merged_clusters').apply(lambda x: x.sample(1)).reset_index(drop=True)
df_filtered.shape[0]

207

In [84]:
df_filtered.to_excel("/content/gdrive/My Drive/Colab Notebooks/Pipeline_Extended/input_ticket_labeling_V2.xlsx")